# Notebook 06: Training a Tiny Transformer

## From Theory to Practice - Building a Language Model

Congratulations on making it to the final notebook! Now we'll bring everything together and train a working transformer. In this notebook:

1. **Character-Level Language Modeling** - Predicting next characters
2. **Training Loop** - Complete training pipeline
3. **Text Generation** - Sampling from the trained model
4. **Training Dynamics** - Monitoring loss, gradients, attention

By the end, you'll have a trained transformer that generates text!

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from typing import Optional, Tuple

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## Part 1: Dataset Preparation

We'll train on a small text corpus (Shakespeare, for tradition!):

In [ ]:
# Sample text (replace with actual data in practice)
SAMPLE_TEXT = """
To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. To die—to sleep,
No more; and by a sleep to say we end
The heart-ache and the thousand natural shocks
That flesh is heir to: 'tis a consummation
Devoutly to be wish'd. To die, to sleep;
To sleep, perchance to dream—ay, there's the rub:
For in that sleep of death what dreams may come,
When we have shuffled off this mortal coil,
Must give us pause—there's the respect
That makes calamity of so long life.
"""

# For real training, download Shakespeare corpus:
try:
    import urllib.request
    url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    with urllib.request.urlopen(url) as response:
        text = response.read().decode('utf-8')
    print(f"✅ Downloaded Shakespeare corpus: {len(text):,} characters")
except:
    text = SAMPLE_TEXT * 100  # Repeat sample if download fails
    print(f"⚠️ Using sample text: {len(text):,} characters")

# Create character vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"\nVocabulary size: {vocab_size}")
print(f"Characters: {''.join(chars[:50])}...")

# Create mappings
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode/decode functions
def encode(text: str) -> list:
    return [char_to_idx[ch] for ch in text]

def decode(indices: list) -> str:
    return ''.join([idx_to_char[i] for i in indices])

# Test encoding
sample = "Hello"
encoded = encode(sample)
decoded = decode(encoded)
print(f"\nTest: '{sample}' → {encoded} → '{decoded}'")

### Character Dataset

In [ ]:
class CharDataset(Dataset):
    """Character-level dataset for next-token prediction."""
    
    def __init__(self, text: str, seq_len: int, char_to_idx: dict):
        self.seq_len = seq_len
        self.char_to_idx = char_to_idx
        self.data = [char_to_idx[ch] for ch in text]
        
    def __len__(self):
        return len(self.data) - self.seq_len
    
    def __getitem__(self, idx):
        # Input: characters [idx:idx+seq_len]
        # Target: characters [idx+1:idx+seq_len+1] (shifted by 1)
        x = torch.tensor(self.data[idx:idx + self.seq_len], dtype=torch.long)
        y = torch.tensor(self.data[idx + 1:idx + self.seq_len + 1], dtype=torch.long)
        return x, y

# Create dataset and dataloader
seq_len = 64
batch_size = 32

dataset = CharDataset(text, seq_len, char_to_idx)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Dataset size: {len(dataset):,} sequences")
print(f"Batch size: {batch_size}")
print(f"Number of batches: {len(dataloader)}")

# Show a sample
x, y = next(iter(dataloader))
print(f"\nSample batch:")
print(f"  Input shape: {x.shape}")
print(f"  Target shape: {y.shape}")
print(f"\nFirst sequence:")
print(f"  Input:  '{decode(x[0].tolist())}'")
print(f"  Target: '{decode(y[0].tolist())}'")

## Part 2: Model Definition

Let's create a small transformer for character-level modeling:

In [ ]:
# Import our transformer components from previous notebook
# (In practice, these would be in a separate module)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        self.W_O = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        Q = self.W_Q(query).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_K(key).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_V(value).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = torch.matmul(attn_weights, V)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.W_O(attn_output)

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.linear2(self.dropout(F.gelu(self.linear1(x))))

class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, n_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention with residual
        attn_out = self.attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_out))
        
        # Feed-forward with residual
        ff_out = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x

class CharTransformer(nn.Module):
    """Character-level transformer language model."""
    
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, 
                 max_len=512, dropout=0.1):
        super().__init__()
        
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(max_len, d_model)
        
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, x, targets=None):
        batch_size, seq_len = x.shape
        
        # Embeddings
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        x = self.token_embedding(x) + self.pos_embedding(positions)
        x = self.dropout(x)
        
        # Create causal mask
        mask = torch.tril(torch.ones(seq_len, seq_len, device=x.device)).view(1, 1, seq_len, seq_len)
        
        # Transformer blocks
        for block in self.blocks:
            x = block(x, mask)
        
        x = self.norm(x)
        logits = self.lm_head(x)
        
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        
        return logits, loss

# Create model
model = CharTransformer(
    vocab_size=vocab_size,
    d_model=128,
    n_layers=4,
    n_heads=4,
    d_ff=512,
    max_len=seq_len,
    dropout=0.1
).to(device)

print(f"✅ Model created")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## Part 3: Training Loop

Let's train the model!

In [ ]:
# Training configuration
learning_rate = 3e-4
max_epochs = 10
eval_interval = 100

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_epochs * len(dataloader))

# Track metrics
train_losses = []
step = 0

print("Starting training...\n")
model.train()

for epoch in range(max_epochs):
    epoch_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{max_epochs}")
    
    for batch_idx, (x, y) in enumerate(progress_bar):
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        logits, loss = model(x, y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        # Track metrics
        epoch_loss += loss.item()
        train_losses.append(loss.item())
        step += 1
        
        # Update progress bar
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}', 'lr': f'{scheduler.get_last_lr()[0]:.2e}'})
    
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1}: Average Loss = {avg_loss:.4f}\n")

print("✅ Training complete!")

In [ ]:
# Plot training loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, alpha=0.3)
# Smooth with moving average
window = 50
if len(train_losses) > window:
    smoothed = np.convolve(train_losses, np.ones(window)/window, mode='valid')
    plt.plot(range(window-1, len(train_losses)), smoothed, linewidth=2, label='Smoothed')
plt.xlabel('Step', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(train_losses[-1000:])  # Last 1000 steps
plt.xlabel('Recent Steps', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Recent Training Loss', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final loss: {train_losses[-1]:.4f}")
print(f"Best loss: {min(train_losses):.4f}")

## Part 4: Text Generation

Now for the fun part - generating text!

In [ ]:
@torch.no_grad()
def generate(model, prompt, max_new_tokens=100, temperature=0.8, top_k=40):
    """
    Generate text from the model.
    
    Args:
        model: Trained transformer
        prompt: Starting text
        max_new_tokens: Number of tokens to generate
        temperature: Sampling temperature (higher = more random)
        top_k: Sample from top-k most likely tokens
    """
    model.eval()
    
    # Encode prompt
    context = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
    
    generated = prompt
    
    for _ in range(max_new_tokens):
        # Crop context if needed
        context_cond = context if context.size(1) <= seq_len else context[:, -seq_len:]
        
        # Get predictions
        logits, _ = model(context_cond)
        logits = logits[:, -1, :] / temperature
        
        # Top-k sampling
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
        
        # Sample
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Append to sequence
        context = torch.cat([context, next_token], dim=1)
        generated += idx_to_char[next_token.item()]
    
    model.train()
    return generated

# Generate samples with different prompts and temperatures
prompts = ["To be", "The ", "What "]
temperatures = [0.5, 0.8, 1.0]

print("="*80)
print("Generated Samples")
print("="*80)

for temp in temperatures:
    print(f"\nTemperature = {temp}")
    print("-"*80)
    for prompt in prompts:
        generated = generate(model, prompt, max_new_tokens=200, temperature=temp, top_k=40)
        print(f"\nPrompt: '{prompt}'")
        print(f"Generated: {generated}")
        print()

## Part 5: Model Analysis

Let's analyze what the model learned:

In [ ]:
# Compute perplexity on validation set
@torch.no_grad()
def compute_perplexity(model, dataloader):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
        total_loss += loss.item() * x.size(0) * x.size(1)
        total_tokens += x.size(0) * x.size(1)
    
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    
    model.train()
    return perplexity, avg_loss

perplexity, avg_loss = compute_perplexity(model, dataloader)
print(f"Model Perplexity: {perplexity:.2f}")
print(f"Average Loss: {avg_loss:.4f}")
print(f"\n💡 Lower perplexity = better model")
print(f"   Random baseline would have perplexity ≈ {vocab_size}")

In [ ]:
# Visualize attention patterns
@torch.no_grad()
def visualize_attention(model, text, layer_idx=0, head_idx=0):
    model.eval()
    
    # Encode text
    tokens = torch.tensor(encode(text), dtype=torch.long, device=device).unsqueeze(0)
    
    # Get embeddings
    positions = torch.arange(0, tokens.size(1), device=device).unsqueeze(0)
    x = model.token_embedding(tokens) + model.pos_embedding(positions)
    
    # Get attention from specific layer and head
    mask = torch.tril(torch.ones(tokens.size(1), tokens.size(1), device=device)).view(1, 1, -1, tokens.size(1))
    
    for i, block in enumerate(model.blocks):
        if i == layer_idx:
            # Extract attention weights
            Q = block.attention.W_Q(x).view(1, -1, block.attention.n_heads, block.attention.d_k).transpose(1, 2)
            K = block.attention.W_K(x).view(1, -1, block.attention.n_heads, block.attention.d_k).transpose(1, 2)
            
            scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(block.attention.d_k)
            scores = scores.masked_fill(mask == 0, -1e9)
            attn_weights = F.softmax(scores, dim=-1)
            
            attn_matrix = attn_weights[0, head_idx].cpu().numpy()
            break
        x = block(x, mask)
    
    # Visualize
    import seaborn as sns
    plt.figure(figsize=(10, 8))
    sns.heatmap(attn_matrix, cmap='viridis', xticklabels=list(text), yticklabels=list(text))
    plt.title(f'Attention Pattern (Layer {layer_idx}, Head {head_idx})', fontsize=14, fontweight='bold')
    plt.xlabel('Keys', fontsize=12)
    plt.ylabel('Queries', fontsize=12)
    plt.show()
    
    model.train()

# Visualize attention on sample text
sample_text = "To be or not to be"
visualize_attention(model, sample_text, layer_idx=0, head_idx=0)

## Part 6: Save and Load Model

In [ ]:
# Save model
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'vocab_size': vocab_size,
    'char_to_idx': char_to_idx,
    'idx_to_char': idx_to_char,
    'config': {
        'd_model': 128,
        'n_layers': 4,
        'n_heads': 4,
        'd_ff': 512,
        'max_len': seq_len
    }
}

torch.save(checkpoint, 'char_transformer.pt')
print("✅ Model saved to 'char_transformer.pt'")

# Load model (example)
# checkpoint = torch.load('char_transformer.pt')
# model = CharTransformer(**checkpoint['config'], vocab_size=checkpoint['vocab_size'])
# model.load_state_dict(checkpoint['model_state_dict'])
# model.to(device)

## Exercise Section

### Exercise 1: Experiment with Architectures
Train models with different configurations:
- Vary number of layers (2, 4, 8)
- Vary d_model (64, 128, 256)
- Vary number of heads

Compare perplexity and generation quality.

In [ ]:
# TODO: Experiment with architectures

### Exercise 2: Implement Beam Search
Instead of sampling, implement beam search:
- Keep top-k sequences at each step
- Score by log probability
- Compare with sampling

In [ ]:
# TODO: Implement beam search

### Exercise 3: Analyze Learned Patterns
Investigate what the model learned:
- Which characters predict which?
- Attention to punctuation
- Long-range dependencies

In [ ]:
# TODO: Analyze learned patterns

## Summary: You've Built a Transformer!

### What You Accomplished

✅ **End-to-End Training:**
- Dataset preparation and batching
- Complete training loop with optimization
- Text generation and evaluation

✅ **Model Architecture:**
- Multi-head self-attention
- Position-wise feed-forward
- Layer normalization and residuals
- Positional embeddings

✅ **Training Techniques:**
- AdamW optimizer
- Learning rate scheduling
- Gradient clipping
- Dropout regularization

✅ **Generation:**
- Temperature sampling
- Top-k filtering
- Autoregressive generation

### Journey Recap

From Notebook 01 to now:
1. **CUDA Basics** - GPU programming fundamentals
2. **Matrix Ops & Softmax** - Neural network primitives
3. **Attention (CPU)** - Understanding the mechanism
4. **Attention (GPU)** - Scaling with hardware
5. **Transformer Block** - Complete architecture
6. **Training** - Working language model!

### Next Steps in Your Journey

**Immediate:**
- Train on larger datasets (full Shakespeare, Wikipedia)
- Experiment with word-level or BPE tokenization
- Try different architectures (GPT-style decoder-only)

**Intermediate:**
- Implement Flash Attention for memory efficiency
- Add more sophisticated sampling (nucleus, typical)
- Fine-tune pre-trained models

**Advanced:**
- Multi-GPU training with DDP
- Model quantization and optimization
- Custom CUDA kernels for specific operations

### Resources for Continued Learning

**Papers:**
- "Attention Is All You Need" (Vaswani et al., 2017)
- "BERT" (Devlin et al., 2018)
- "GPT-3" (Brown et al., 2020)
- "Flash Attention" (Dao et al., 2022)

**Code:**
- HuggingFace Transformers library
- Andrej Karpathy's nanoGPT
- Fairseq (Facebook AI)

## Further reading (Archive.org)

To place this tiny Transformer training loop in context, use Archive.org searches such as:

- "sequence to sequence learning tutorial"
- "language modeling deep learning"
- "character level language model"

Prioritize lecture notes or book chapters that describe next-token prediction, cross-entropy loss, and teacher forcing, as they closely match the toy training setup and sampling procedure implemented here.

---

## Congratulations! 🎉

You've completed the CUDA Transformer tutorial. You now understand:
- How GPUs accelerate deep learning
- The mathematics behind transformers
- How to implement and train modern architectures

**Keep building, keep learning, and enjoy your journey in deep learning!** 🚀